In [1]:
import openmeteo_requests
import matplotlib as plt
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime as dt
from datetime import timedelta

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 47.26580883196723,
	"longitude": 11.84457426992035,
    'past_days': 7,
    'forecast_days': 0,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_120m", "wind_direction_10m", "wind_direction_120m", "wind_gusts_10m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_temperature_18cm", "soil_temperature_54cm", "soil_moisture_0_to_1cm", "soil_moisture_1_to_3cm", "soil_moisture_3_to_9cm", "soil_moisture_9_to_27cm", "soil_moisture_27_to_81cm"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 47.2400016784668°N 11.84000015258789°E
Elevation 1818.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [3]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_snow_depth = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                         date  snow_depth
0   2024-12-10 00:00:00+00:00        0.24
1   2024-12-10 01:00:00+00:00        0.24
2   2024-12-10 02:00:00+00:00        0.24
3   2024-12-10 03:00:00+00:00        0.24
4   2024-12-10 04:00:00+00:00        0.24
..                        ...         ...
163 2024-12-16 19:00:00+00:00        0.20
164 2024-12-16 20:00:00+00:00        0.20
165 2024-12-16 21:00:00+00:00        0.20
166 2024-12-16 22:00:00+00:00        0.20
167 2024-12-16 23:00:00+00:00        0.19

[168 rows x 2 columns]


In [4]:
# Load the data
st.title("Interactive Snow Depth Visualization")
st.write("This interactive line graph shows the past snow depth over the last 7 days.")

# Plot with Plotly
fig = px.line(
    data,
    x='date',
    y='snow_depth',
    title="Snow Depth Over Time",
    labels={"snow_depth": "Snow Depth (cm)", "date": "Date"},
    markers=True
)
st.plotly_chart(fig)

NameError: name 'st' is not defined